## Setup

In [3]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2015, day=4)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [4]:
# Import performance checking utility
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_example, time_solution

## Part a

In [5]:
# Imports
from hashlib import md5

In [6]:
# Functions
def solve_a(input_data: str, *, num_zeroes: int = 5) -> int:
    """Find the lowest integer `i` > 0 such that md5("input_data"+"i") starts with n zeroes."""
    target_hash_prefix = "0" * num_zeroes

    for i in range(1, int(1e9)):
        hash_input = f"{input_data}{i}".encode()
        hash_output = md5(hash_input).hexdigest()  # noqa: S324 # MD5 is not used for security purposes here
        if hash_output.startswith(target_hash_prefix):
            return i

    msg = "No solution found within the first 1 billion integers."
    raise RuntimeError(msg)

In [7]:
# Correctness check
check_example(solve_a, example)

solve_a found answer 609043, which is the correct solution for part A!


True

In [16]:
# Performance check
time_a = time_solution(solve_a, input_data, iterations=5)

solve_a takes 136.89 ms


In [20]:
# Submit answer
puzzle.answer_a = solve_a(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine. [Continue to Part Two]


## Part b

### Simple approach
We can simply use the same logic and adjust the number of leading zeroes required in the hash from 5 to 6.

In [ ]:
# Performance check
time_b_simple = time_solution(
    solve_a, input_data, num_zeroes=6, iterations=5, runs=2, time_unit="s", print_result=False
)
print(f"Finding a hash with 6 leading zeroes takes {time_b_simple:.2f} seconds.")

Finding a hash with 6 leading zeroes takes 5.11 seconds.


### Parallelized approach

Hasing in one thread takes 5 seconds. Let's see if we can speed things up with parallel threads.

In [3]:
# Imports
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Manager
from os import cpu_count

from _4_worker import worker_b

In [17]:
def solve_b_parallel(input_data: str, *, num_zeroes: int = 6, worker_count: int | None = None) -> int:
    """Find the lowest integer `i` > 0 such that md5("input_data"+"i") starts with n zeroes, using multiple threads."""
    if worker_count is None:
        # Leave one core free
        worker_count = (cpu_count() or 4) - 1

    # Create a stop event to signal workers to stop
    stop_event = Manager().Event()

    with ProcessPoolExecutor(max_workers=worker_count) as ex:
        # Create and submit worker tasks
        futures = [
            ex.submit(
                worker_b, input_data, num_zeroes=num_zeroes, start=start, step=worker_count, stop_event=stop_event
            )
            for start in range(1, worker_count + 1)
        ]

        # Collect results as they complete, and return the first one found
        for fut in as_completed(futures):
            if (result := fut.result()) is not None:
                return result

    msg = "No result found"
    raise RuntimeError(msg)

In [18]:
# Performance check
time_b_parallel = time_solution(solve_b_parallel, input_data, iterations=5, runs=2, time_unit="s")
print(f"This is {time_b_simple / time_b_parallel:.2f} times faster than the single-threaded version.")

solve_b_parallel takes 1.38 s
This is 3.70 times faster than the single-threaded version.


Great! Spreading the work across multiple processes speeds things up by around 3.5x on my machine with 7 cores available.

In [ ]:
# Submit answer
puzzle.answer_b = solve_b_parallel(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine.You have completed Day 4! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
